In [1]:
#inicializar kernel

In [6]:
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine

## Funciones
### Creamos una funcion para descargar los datos de archivos excel donde podemos especificar las hojas a descargar, otra para analizar valores y una tercera para limpiar los datos

In [13]:
def cargar_datos_desde_excel(archivo, hojas, engine='openpyxl'):
    """
    Carga datos desde un archivo Excel y devuelve un diccionario de DataFrames.

    Parameters:
    - archivo (str): Ruta del archivo Excel.
    - hojas (list): Lista de nombres de hojas a cargar.
    - engine (str, optional): Motor de Excel a utilizar. Por defecto, 'openpyxl'.

    Returns:
    dfs: Un diccionario donde las claves son los nombres de las hojas y los valores son DataFrames correspondientes.

    Example:
    >>> datos = cargar_datos_desde_excel('archivo.xlsx', ['Hoja1', 'Hoja2'])
    >>> df_hoja1 = datos['Hoja1']
    >>> df_hoja2 = datos['Hoja2']
    """
    
    xls_file = pd.ExcelFile(archivo, engine=engine)
    dfs = {}

    for hoja in hojas:
        df = pd.read_excel(xls_file, hoja) 
        dfs[hoja] = df

    return dfs


In [14]:
def analizar_valores_sd(dataframe):
    """
    Analiza la presencia de valores 'SD' en cada columna del DataFrame.

    Parameters:
    dataframe (pd.DataFrame): El DataFrame a analizar.

    Returns:
    pd.DataFrame: Un DataFrame que muestra la cantidad y porcentaje de valores 'SD' en cada columna.
    """
    columnas_con_sd = dataframe.columns
    resultados = []

    for columna in columnas_con_sd:
        cantidad_sd = dataframe[columna].eq('SD').sum()
        porcentaje_sd = (cantidad_sd / len(dataframe)) * 100
        resultados.append({'Columna': columna, 'Cantidad de SD': cantidad_sd, 'Porcentaje de SD': porcentaje_sd})

    resultados_df = pd.DataFrame(resultados)
    resultados_con_sd = resultados_df[resultados_df['Cantidad de SD'] > 0]

    return resultados_con_sd

In [15]:
def data_cleaning(df, drop_duplicates=False, drop_na=False, fill_na=None, convert_to_datetime=None, uppercase_columns=None,
                  lowercase_columns=None, titlecase_columns=None, strip_spaces=True, rename_columns=None, drop_columns=None,
                  categorize_columns=None, replace_values=None, new_columns=None, convert_date_columns=None, 
                  convert_to_int_columns=None, convert_to_float=None, new_columns2=None):
    """
    Realiza el proceso de limpieza de datos en un DataFrame.

    Parámetros:
    - df (pd.DataFrame): El DataFrame que se va a limpiar.
    
    - drop_duplicates (bool): Elimina duplicados si es True.
      Ejemplo: cleaned_df = data_cleaning(df_tu_data_frame, drop_duplicates=True)
    
    - drop_na (bool): Elimina filas con valores nulos si es True.
      Ejemplo: cleaned_df = data_cleaning(df_tu_data_frame, drop_na=True)

    - fill_na (dict): Un diccionario donde las claves son los nombres de columnas y los valores son valores para rellenar los nulos.
      Ejemplo: fill_na_dict = {'gravedad': 'leve'}
               cleaned_df = data_cleaning(df_tu_data_frame, fill_na=fill_na_dict)
        
    - convert_to_datetime (list): Lista de columnas para convertir a tipo de dato datetime.
      Ejemplo: columns_to_convert = ['fecha', 'hora']
               cleaned_df = data_cleaning(df_tu_data_frame, convert_to_datetime=columns_to_convert)

    - uppercase_columns (list): Lista de columnas para convertir a mayúsculas.
      Ejemplo: columns_to_uppercase = ['nombre', 'apellido']
               cleaned_df = data_cleaning(df_tu_data_frame, uppercase_columns=columns_to_uppercase)

    - lowercase_columns (list): Lista de columnas para convertir a minúsculas.
      Ejemplo: columns_to_lowercase = ['Ciudad', 'Pais']
               cleaned_df = data_cleaning(df_tu_data_frame, lowercase_columns=columns_to_lowercase)

    - titlecase_columns (list): Lista de columnas para convertir a formato de título (primera letra en mayúscula, resto en minúscula).
      Ejemplo: columns_to_titlecase = ['titulo', 'categoria']
               cleaned_df = data_cleaning(df_tu_data_frame, titlecase_columns=columns_to_titlecase)
            
    - strip_spaces (bool): Elimina espacios en blanco alrededor de los valores de las celdas si es True.
      Ejemplo: cleaned_df = data_cleaning(df_tu_data_frame, strip_spaces=True)

    - rename_columns (dict): Un diccionario donde las claves son los nombres de las columnas actuales y los valores son los nuevos nombres.
      Ejemplo: rename_dict = {'Vieja_Columna': 'Nueva_Columna'}
               cleaned_df = data_cleaning(df_tu_data_frame, rename_columns=rename_dict)
    
    - drop_columns (list): Lista de columnas para eliminar.
      Ejemplo: columns_to_drop = ['columna1', 'columna2']
               cleaned_df = data_cleaning(df_tu_data_frame, drop_columns=columns_to_drop)
        
    - categorize_columns (list): Lista de columnas para convertir a tipo de dato categoría.
      Ejemplo: columns_to_categorize = ['categoria1', 'categoria2']
               cleaned_df = data_cleaning(df_tu_data_frame, categorize_columns=columns_to_categorize)
        
    - replace_values (dict): Un diccionario donde las claves son los nombres de las columnas y los valores son diccionarios de reemplazo.
      Ejemplo: replace_dict = {'columna1': {'Antiguo1': 'Nuevo1', 'Antiguo2': 'Nuevo2'}}
               cleaned_df = data_cleaning(df_tu_data_frame, replace_values=replace_dict)

    - new_columns (dict): Un diccionario donde las claves son los nombres de las nuevas columnas y los valores son valores para esas columnas.
      Ejemplo: new_columns_dict = {'nueva_columna': 0}
               cleaned_df = data_cleaning(df_tu_data_frame, new_columns=new_columns_dict)
               
    - new_columns2 (dict): Un diccionario donde las claves son los nombres de las nuevas columnas y los valores son expresiones
                          para calcular el contenido de las nuevas columnas basadas en otras columnas existentes. 
      Ejemplo: {'nueva_columna1': 'columna_existente * 2'}
      cleaned_df = data_cleaning(df_tu_data_frame, new_columns2=new_columns_dict)

 
    - convert_date_columns (dict): Un diccionario donde las claves son los nombres de las columnas y los valores son los formatos de fecha.
      Ejemplo: date_columns_dict = {'fecha': '%Y-%m-%d', 'hora': '%H:%M:%S'}
               cleaned_df = data_cleaning(df_tu_data_frame, convert_date_columns=date_columns_dict)

    - convert_to_int_columns (list): Lista de columnas para convertir a tipo de dato entero.
      Ejemplo: columns_to_int = ['columna1', 'columna2']
               cleaned_df = data_cleaning(df_tu_data_frame, convert_to_int_columns=columns_to_int)
    
    - convert_to_float (list): Lista de columnas para convertir a tipo de dato float.
      Ejemplo: columns_to_float = ['columna1', 'columna2']
               cleaned_df = data_cleaning(df_tu_data_frame, convert_to_float=columns_to_float)          
            
    Retorna:
    pd.DataFrame: El DataFrame limpio.
    """

    cleaned_df = df.copy()

    # Eliminar duplicados
    if drop_duplicates:
        cleaned_df.drop_duplicates(inplace=True)
        

    # Eliminar filas con valores nulos
    if drop_na:
        cleaned_df.dropna(inplace=True)
        

    # Rellenar valores nulos
    if fill_na:
        cleaned_df.fillna(fill_na, inplace=True)

        
    # Convertir columnas a tipo datetime
    if convert_to_datetime:
        for column in convert_to_datetime:
            cleaned_df[column] = pd.to_datetime(cleaned_df[column], errors='coerce')
            

    # Convertir columnas a mayúsculas
    if uppercase_columns:
        for column in uppercase_columns:
            cleaned_df[column] = cleaned_df[column].str.upper()
            

    # Convertir columnas a minúsculas
    if lowercase_columns:
        for column in lowercase_columns:
            cleaned_df[column] = cleaned_df[column].str.lower()
        

    # Convertir columnas a formato de título
    if titlecase_columns:
        for column in titlecase_columns:
            cleaned_df[column] = cleaned_df[column].str.title()
            
            
    # Tratar columnas con espacios
    if strip_spaces:
        cleaned_df = cleaned_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        

    # Renombrar columnas
    if rename_columns:
        cleaned_df.rename(columns=rename_columns, inplace=True)
        
    
    # Eliminar columnas
    if drop_columns:
        cleaned_df.drop(columns=drop_columns, inplace=True)
        
        
    # Categorizar columnas
    if categorize_columns:
        for column in categorize_columns:
            if column in cleaned_df.columns:
                cleaned_df[column] = cleaned_df[column].astype('category')
            else:
                print(f"La columna '{column}' no existe en el DataFrame.")
                

    # Reemplazar valores en columnas
    if replace_values:
        for column, replacements in replace_values.items():
            cleaned_df[column].replace(replacements, inplace=True)
            
    # Agregar nuevas columnas
    if new_columns:
        for column, value in new_columns.items():
            cleaned_df[column] = value
            
    # Agregar nuevas columnas basadas en otras columnas
    if new_columns2:
        for new_column, column_expr in new_columns2.items():
            # Verificar si la expresión es proporcionada
            if column_expr:
                cleaned_df[new_column] = cleaned_df.eval(column_expr)
            else:
                cleaned_df[new_column] = None  # O cualquier valor predeterminado que prefieras
                  
    # Convertir columnas de fecha con formato específico
    if convert_date_columns:
        for column, date_format in convert_date_columns.items():
            cleaned_df[column] = pd.to_datetime(cleaned_df[column], format=date_format, errors='coerce')

    
    # Convertir columnas a tipo de dato entero
    if convert_to_int_columns:
        for column in convert_to_int_columns:
            cleaned_df[column] = pd.to_numeric(cleaned_df[column], errors='coerce').astype('Int64')
    
    
    # Convertir columnas a tipo float
    if convert_to_float:
        for column in convert_to_float:
            cleaned_df[column] = cleaned_df[column].astype(float)
        
            
    return cleaned_df

## Empesamos a descargar y visualizar los datos de la Hoja HECHOS

In [9]:
datos_homicidios = cargar_datos_desde_excel('homicidios.xlsx', ['HECHOS', 'VICTIMAS'])

In [10]:
h_hechos_df = datos_homicidios['HECHOS']
print("Original DataFrame:")
h_hechos_df.head(3)

Original DataFrame:


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [11]:
h_hechos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Obsevamos parametros similares que luego en la etapa de Eda los revisaremos mas minusiosamente

In [16]:
resultados_h_hechos = analizar_valores_sd(h_hechos_df)
resultados_h_hechos

,Columna,Cantidad de SD,Porcentaje de SD
6,HORA,1,0.143678
7,HH,1,0.143678
8,LUGAR_DEL_HECHO,1,0.143678
19,VICTIMA,9,1.293103
20,ACUSADO,23,3.304598


In [17]:
# Reemplazar 'SD' por NaN en todo el DataFrame

h_hechos_df.replace(['SD','sd'], np.nan, inplace=True)

In [18]:
# Columnas para pasar a minúcula

columns_to_lower = ['TIPO_DE_CALLE', 'VICTIMA', 'ACUSADO']

In [19]:
# Columnas para eliminar

columns_to_drop = ['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO','Calle','Altura',
                   'Cruce','Dirección Normalizada','XY (CABA)','PARTICIPANTES']

Decidimos eliminar las columnas 'AAAA', 'MM', y 'DD' del conjunto de datos, ya que la información de fecha ya está presente en la columna 'FECHA'. Esta decisión simplifica el conjunto de datos, mantiene una estructura más consistente y estandarizada, ahorra espacio y facilita el análisis temporal.

Asimismo, eliminamos la columna 'HORA' debido a la redundancia de la información de hora, que ya está contenida en la columna 'HH'. Esta elección nos proporciona una visión más general y simplificada de la distribución de los siniestros a lo largo del día.

Las columnas 'LUGAR_DEL_HECHO','Calle','Altura','Cruce' y 'Dirección Normalizada' implicaban datos de la dirección del hecho incluso, 'XY (CABA)' que contenía coordenadas en formato de proyección cartesiana también fue eliminada. Preferimos trabajar con las columnas de latitud y longitud por su mayor intuición, no tiene datos nulos, familiaridad para la mayoría de las personas, facilidad de representación gráfica en mapas y conformidad con estándares de sistemas de información geográfica (SIG).

La columna 'PARTICIPANTES' contenia información concatenada redundante de 'VICTIMA' y 'ACUSADO', así que se mantuvieron éstas dos últimas.

In [20]:
rename_dict = {'ID': 'id_siniestro', 'N_VICTIMAS': 'nro_victimas', 'FECHA': 'fecha', 'HH': 'franja_hora',
               'TIPO_DE_CALLE': 'tipo_calle', 'COMUNA': 'comuna', 'pos x': 'longitud', 'pos y': 'latitud',
               'VICTIMA': 'vehiculo_victima', 'ACUSADO': 'vehiculo_acusado'}

In [21]:
# Convertir a entero

columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']

In [22]:
# Categorización de columnas

columns_to_categorize = ['franja_hora', 'tipo_calle', 'comuna', 'vehiculo_victima', 'vehiculo_acusado']

In [23]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

h_hechos_dfcleaned = data_cleaning(h_hechos_df,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                   )

C:\Users\MAQUINA\AppData\Local\Temp\ipykernel_10116\2722711488.py:125: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = cleaned_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [25]:
h_hechos_dfcleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_siniestro      696 non-null    object        
 1   nro_victimas      696 non-null    Int64         
 2   fecha             696 non-null    datetime64[ns]
 3   franja_hora       695 non-null    Int64         
 4   tipo_calle        696 non-null    category      
 5   comuna            696 non-null    Int64         
 6   longitud          696 non-null    object        
 7   latitud           696 non-null    object        
 8   vehiculo_victima  687 non-null    category      
 9   vehiculo_acusado  673 non-null    category      
dtypes: Int64(3), category(3), datetime64[ns](1), object(3)
memory usage: 43.2+ KB


## Empesamos a descargar y visualizar los datos de la Hoja VICTIMAS

In [26]:
# Acceder al DataFrame por nombre de hoja, Homicidios - Víctimas

h_victimas_df = datos_homicidios['VICTIMAS']

In [27]:
print("Original DataFrame:")
h_victimas_df.head(3)

Original DataFrame:


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00


In [28]:
# Obtener información general del DataFrame

h_victimas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [29]:
# Invoca la función 'analizar_valores_sd'

resultados_h_victimas = analizar_valores_sd(h_victimas_df)
resultados_h_victimas

,Columna,Cantidad de SD,Porcentaje de SD
5,ROL,11,1.534170
6,VICTIMA,9,1.255230
7,SEXO,6,0.836820
8,EDAD,53,7.391911
9,FECHA_FALLECIMIENTO,68,9.483961


In [30]:
# Columnas para pasar a minúcula

columns_to_lower = ['SEXO','ROL']

In [31]:
# Columnas para eliminar

columns_to_drop = ['FECHA','AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO', 'VICTIMA']

Las columnas que decidimos eliminar nos parecen irrelevantes para nuestro análisis. Algunas de ellas se complementan con los datos de la hoja 'HECHO'

In [32]:
# Renombrar columnas

rename_dict = {'ID_hecho': 'id_siniestro', 'SEXO': 'sexo',
               'EDAD': 'edad', 'ROL': 'rol'}

In [33]:
# Convertir a entero

columns_to_int = ['edad']

In [34]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

h_victimas_dfcleaned = data_cleaning(h_victimas_df,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    )

C:\Users\MAQUINA\AppData\Local\Temp\ipykernel_10116\2722711488.py:125: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = cleaned_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [35]:
# Categorización de columnas

columns_to_categorize = ['sexo','edad','rol']
h_victimas_dfcleaned = data_cleaning(h_victimas_dfcleaned, categorize_columns=columns_to_categorize) 

C:\Users\MAQUINA\AppData\Local\Temp\ipykernel_10116\2722711488.py:125: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = cleaned_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [36]:
print("\nCleaned DataFrame:")
h_victimas_dfcleaned.head()


Cleaned DataFrame:


,id_siniestro,rol,sexo,edad
0,2016-0001,conductor,masculino,19
1,2016-0002,conductor,masculino,70
2,2016-0003,conductor,masculino,30
3,2016-0004,conductor,masculino,18
4,2016-0005,conductor,masculino,29


In [38]:
# Fusionar DataFrames

merged_df = pd.merge(h_hechos_dfcleaned, h_victimas_dfcleaned, left_on='id_siniestro', right_on='id_siniestro', how='inner')

In [41]:
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_siniestro      717 non-null    object        
 1   nro_victimas      717 non-null    Int64         
 2   fecha             717 non-null    datetime64[ns]
 3   franja_hora       716 non-null    Int64         
 4   tipo_calle        717 non-null    category      
 5   comuna            717 non-null    Int64         
 6   longitud          717 non-null    object        
 7   latitud           717 non-null    object        
 8   vehiculo_victima  708 non-null    category      
 9   vehiculo_acusado  694 non-null    category      
 10  rol               717 non-null    category      
 11  sexo              717 non-null    category      
 12  edad              664 non-null    category      
dtypes: Int64(3), category(6), datetime64[ns](1), object(3)
memory usage: 49.6+ KB


In [42]:
# Almacenar los DataFrames finales en un nuevo archivo CSV

merged_df.to_csv('homicidios_cleaned.csv', index=False)